In [51]:
import numpy as np
import pandas as pd
import antropy as ent
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.fft import fft, fftfreq 
import seaborn as sns
from scipy import signal
import seaborn as sns
from scipy import stats
from os.path import exists

sns.set_context('paper')

# Definiendo las funciones requeridas

In [62]:

# Gráfica de la evolución de la entropía para diferentes subjects
def plotEntropyAnalisis(subjects, phaseslist, ent_kind, metrics, step, overlap):
    bandnames = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'all bands']
    print("loading the data")
    # we load the data
    for i,subject in enumerate(subjects):
        phases = phaseslist[i]
        comparison = phases[0]+phases[1]
        for metric in metrics:
            fileroot = 'results/entropy_'+subject+'_'+ent_kind+'_'+str(metric)+'_'+str(step)+'_'+str(overlap)
            # we load the entropy information
            ent = pd.read_excel(fileroot+'_ent.xlsx').replace([np.inf, -np.inf], np.nan).dropna()
            ent['metric'] = metric
            ent['subject'] = subject
            # we load the pre-pre comparison information
            stat = pd.read_excel(fileroot+'_'+phases[0]+'_stats.xlsx')
            mask = pd.notna(stat['diff'])
            stat = stat[mask]
            stat['metric'] = metric
            stat['subject'] = subject
            # we load the pre-post comparison information
            stat_prepost = pd.read_excel(fileroot+'_'+comparison+'_stats.xlsx')
            mask = pd.notna(stat_prepost['diff'])
            stat_prepost = stat_prepost[mask]
            stat_prepost['metric'] = metric
            stat_prepost['subject'] = subject
            # we load the first-last comparison information
            stat_firstlast = pd.read_excel(fileroot+'_firstlast_stats.xlsx')
            mask = pd.notna(stat_firstlast['diff'])
            stat_firstlast = stat_firstlast[mask]
            stat_firstlast['metric'] = metric
            stat_firstlast['subject'] = subject
            try:
                ent_data = pd.concat([ent_data, ent])
                stat_data = pd.concat([stat_data, stat])
                stat_prepost_data = pd.concat([stat_prepost_data, stat_prepost])
                stat_firstlast_data = pd.concat([stat_firstlast_data, stat_firstlast])
            except:
                ent_data = ent
                stat_data = stat
                stat_prepost_data = stat_prepost
                stat_firstlast_data = stat_firstlast
    # creating a new column with the subject + session information
    ent_data['info'] = ent_data['subject']+'_'+ent_data['sesion'].astype('str')+'_'+ent_data['fase']
    stat_data['info'] = stat_data['subject']+'_'+stat_data['sesion'].astype('str')
    stat_prepost_data['info'] = stat_prepost_data['subject']+'_'+stat_prepost_data['sesion'].astype('str')


    print("loaded the data")    
    
    # now we will add some additional columns to work on the plots
   # test = stat_data.merge(stat_data.groupby(['eeg','bands','subject','metric'])['diff'].mean().reset_index().rename({'diff':'max_diff'}, axis = 1), on=['eeg','bands','subject','metric'], how = 'left')
   # print(test.head())
    
    
    # A figure will be created for each band and each EEG. In each case a figure showing the evolution of the calculated
    # entropy as a function of the used metric. The session number, or control number, will be used as the hue
    
    
    for band in bandnames:
        for eeg in ['EEG1', 'EEG2']:
            ###################################################################################
            ################## GENERATING THE PLOT
            if subjects[-1]!='C':
                fig = plt.figure(figsize=(10,20))
                spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                spec.update(left=0.05, right= 0.99, top = 0.95, bottom = 0.52, wspace=0.1, hspace = 0.25)
                spec2 = gridspec.GridSpec(nrows = 3, ncols=1, figure=fig)
                spec2.update(left=0.05, right= 0.99, top = 0.46, bottom = 0.03, wspace=0.1, hspace = 0.25)
            else:
                fig = plt.figure(figsize=(10,8))
                spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                spec.update(left=0.05, right= 0.99, top = 0.9, bottom = 0.1, wspace=0.1, hspace = 0.25)

                

            line1 = []
            line2 = []
            line3 = []
            line4 = []
            line5 = []
            line1.append(plt.subplot(spec[0,0]))
            line2.append(plt.subplot(spec[1,0], sharey = line1[0]))
            if subjects[-1]!='C':
                line3.append(plt.subplot(spec2[0,0]))
                line4.append(plt.subplot(spec2[1,0], sharex = line3[0]))
                line5.append(plt.subplot(spec2[2,0]))


            ax = [line1,line2,line3,line4,line5]
  
            #ax.set_title('Evolución entropía\n modo: '+key+', señal: '+eeg)
            if ent_kind == 'sampen':
                fig.suptitle('Sample Entropy; band: '+band+'; eeg: '+eeg)
            elif ent_kind == 'permen':
                fig.suptitle('Permutation Entropy; band: '+band+'; eeg: '+eeg)

            for i, fase in enumerate(['pre', 'post']):
                mask = ((ent_data['fase']==fase) & (ent_data['eeg']==eeg) & (ent_data['banda']==band)) | ((ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['subject']=='C'))
                sns.boxplot(data = ent_data[mask], x='metric', y='entropia', hue ='info', ax = ax[i][0])
                ax[i][0].legend(fontsize=7, loc = 'upper right')
                ax[i][0].set_title("Evolución Entropía "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band + '; fase: '+ fase)

            subnames = '_'.join(subjects)
            if subjects[-1]!='C':                
                # plotting the pre -pre changes
                mask = (stat_data['eeg']==eeg) & (stat_data['bands']==band) & (stat_data['subject']!='C')
                sns.barplot(data = stat_data[mask], x='metric', y='diff', hue = 'sesion', ax = ax[2][0])
                ax[2][0].axhline(y=0)
                ax[2][0].legend(fontsize=7, loc = 'upper left')
                ax[2][0].set_title("cambio "+ subject[-1] +" PRE-PRE "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)
                
                # plotting the pre-post changes
                mask = (stat_prepost_data['eeg']==eeg) & (stat_prepost_data['bands']==band) & (stat_prepost_data['subject']!='C')
                sns.barplot(data = stat_prepost_data[mask], x='metric', y='diff', hue = 'sesion', ax = ax[3][0])
                ax[3][0].axhline(y=0)
                ax[3][0].legend(fontsize=7, loc = 'upper left')
                ax[3][0].set_title("cambio "+ subject[-1] +" PRE-POST "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)
                
                # plotting the first-last changes
                mask = (stat_firstlast_data['eeg']==eeg) & (stat_firstlast_data['bands']==band) & (stat_firstlast_data['subject']!='C')
                sns.scatterplot(data = stat_firstlast_data[mask], x='metric', y='diff', hue = 'fase', style = 'signif', ax = ax[4][0])
                ax[4][0].axhline(y=0)
                ax[4][0].legend(fontsize=7, loc = 'upper left')
                ax[4][0].set_title("cambio "+ subject[-1] +" PRIMERO-ULTIMO "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)

            filename = 'images/entropy_'+subnames+'_'+ent_kind+'_'+eeg+'_'+band+'_'+str(overlap)+'.png'
            plt.savefig(filename, dpi = 400)
            plt.close()
    
            
# Gráfica de la evolución de la entropía para diferentes subjects
def plotEntropyAnalisisMS(subjects, phaseslist, metrics, scales, step, overlap):
    ent_kind = 'msen'
    bandnames = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'all bands']
    # we load the data
    for i,subject in enumerate(subjects):
        phases = phaseslist[i]
        comparison = phases[0]+phases[1]
        for metric in metrics:
            for scale in scales:
                print(metric,scale)
                fileroot = 'results/entropy_'+subject+'_'+ent_kind+'_'+str(metric)+'_'+str(scale)+'_'+str(step)+'_'+str(overlap)
                ent = pd.read_excel(fileroot+'_ent.xlsx')
                ent['metric'] = metric
                ent['subject'] = subject
                ent['scale'] = scale
               # print(len(ent))
                stat = pd.read_excel(fileroot+'_'+phases[0]+'_stats.xlsx')
                mask = pd.notna(stat['diff'])
                stat = stat[mask]
                stat['metric'] = metric
                stat['subject'] = subject
                stat['scale'] = scale
               # print(len(stat))
                stat_prepost = pd.read_excel(fileroot+'_'+comparison+'_stats.xlsx')
                mask = pd.notna(stat_prepost['diff'])
                stat_prepost = stat_prepost[mask]
                stat_prepost['metric'] = metric
                stat_prepost['subject'] = subject
                stat_prepost['scale'] = scale
                # we load the first-last comparison information
                stat_firstlast = pd.read_excel(fileroot+'_firstlast_stats.xlsx')
                mask = pd.notna(stat_firstlast['diff'])
                stat_firstlast = stat_firstlast[mask]
                stat_firstlast['metric'] = metric
                stat_firstlast['subject'] = subject
                stat_firstlast['scale'] = scale
                #print(len(stat_prepost))
                try:
                    ent_data = pd.concat([ent_data, ent])
                    stat_data = pd.concat([stat_data, stat])
                    stat_prepost_data = pd.concat([stat_prepost_data, stat_prepost])
                    stat_firstlast_data = pd.concat([stat_firstlast_data, stat_firstlast])

                except:
                    ent_data = ent
                    stat_data = stat
                    stat_prepost_data = stat_prepost
                    stat_firstlast_data = stat_firstlast
    # creating a new column with the subject + session information
    ent_data['info'] = ent_data['subject']+'_'+ent_data['sesion'].astype('str')+'_'+ent_data['fase']
    stat_data['info'] = stat_data['subject']+'_'+stat_data['sesion'].astype('str')
    stat_prepost_data['info'] = stat_prepost_data['subject']+'_'+stat_prepost_data['sesion'].astype('str')

                    
    print("loaded the data")
    # A figure will be created for each band, each EEG, and each metric. In each case a figure showing the evolution of the calculated
    # entropy as a function of the used metric. The session number, or control number, will be used as the hue
    
    for metric in metrics:
        for band in bandnames:
            for eeg in ['EEG1', 'EEG2']:
                ###################################################################################
                ################## GENERATING THE PLOT
                if subjects[-1]!='C':
                    fig = plt.figure(figsize=(10,20))
                    spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                    spec.update(left=0.05, right= 0.99, top = 0.95, bottom = 0.52, wspace=0.1, hspace = 0.25)
                    spec2 = gridspec.GridSpec(nrows = 3, ncols=1, figure=fig)
                    spec2.update(left=0.05, right= 0.99, top = 0.46, bottom = 0.03, wspace=0.1, hspace = 0.25)
                else:
                    fig = plt.figure(figsize=(10,8))
                    spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                    spec.update(left=0.05, right= 0.99, top = 0.9, bottom = 0.1, wspace=0.1, hspace = 0.25)



                line1 = []
                line2 = []
                line3 = []
                line4 = []
                line5 = []
                line1.append(plt.subplot(spec[0,0]))
                line2.append(plt.subplot(spec[1,0], sharey = line1[0]))
                if subjects[-1]!='C':
                    line3.append(plt.subplot(spec2[0,0]))
                    line4.append(plt.subplot(spec2[1,0], sharex = line3[0]))
                    line5.append(plt.subplot(spec2[2,0]))


                ax = [line1,line2,line3,line4,line5]

                #ax.set_title('Evolución entropía\n modo: '+key+', señal: '+eeg)
                fig.suptitle('Multiscale Entropy; band: '+band+'; eeg: '+eeg+'; metric: '+ str(metric))


                for i, fase in enumerate(['pre', 'post']):
                    mask = ((ent_data['fase']==fase) & (ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['metric']==metric)) | ((ent_data['metric']==metric) & (ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['subject']=='C'))
                    sns.boxplot(data = ent_data[mask], x='scale', y='entropia', hue ='info', ax = ax[i][0])
                    ax[i][0].legend(fontsize=7, loc = 'upper right')
                    ax[i][0].set_title("Evolución Entropía "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band + '; fase: '+ fase)

                subnames = '_'.join(subjects)
                if subjects[-1]!='C':                
                    mask = (stat_data['eeg']==eeg) & (stat_data['bands']==band) & (stat_data['metric']==metric) & (stat_data['subject']!='C')
                    sns.barplot(data = stat_data[mask], x='scale', y='diff', hue = 'sesion', ax = ax[2][0])
                    ax[2][0].axhline(y=0)
                    ax[2][0].legend(fontsize=7, loc = 'upper left')
                    ax[2][0].set_title("cambio "+ subject[-1] +" PRE-PRE "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)
                    #sns.boxplot(data = statsPREPOST[mask], x='bands', y='diff', hue = 'sesion', ax = ax[j][3])
                    #ax[j][3].legend(fontsize=4)
                    mask = (stat_prepost_data['eeg']==eeg) & (stat_prepost_data['bands']==band) & (stat_prepost_data['subject']!='C') & (stat_prepost_data['metric']==metric)
                    sns.barplot(data = stat_prepost_data[mask], x='scale', y='diff', hue = 'sesion', ax = ax[3][0])
                    ax[3][0].axhline(y=0)
                    ax[3][0].legend(fontsize=7, loc = 'upper left')
                    ax[3][0].set_title("cambio "+ subject[-1] +" PRE-POST "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band) 
                     # plotting the first-last changes
                    mask = (stat_firstlast_data['eeg']==eeg) & (stat_firstlast_data['bands']==band) & (stat_firstlast_data['subject']!='C') & (stat_firstlast_data['metric']==metric)
                    sns.scatterplot(data = stat_firstlast_data[mask], x='scale', y='diff', hue = 'fase', style = 'signif', ax = ax[4][0])
                    ax[4][0].axhline(y=0)
                    ax[4][0].legend(fontsize=7, loc = 'upper left')
                    ax[4][0].set_title("cambio "+ subject[-1] +" PRIMERO-ULTIMO "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)

                filename = 'images/entropy_'+subnames+'_'+ent_kind+'_'+eeg+'_'+band+'_'+str(metric)+'_'+str(overlap)+'.png'
                plt.savefig(filename, dpi = 400)
                plt.close()

# Plotting

## Control evolution

## Evolución Pacientes

In [65]:
Plist = np.arange(7)+1
print(Plist)

[1 2 3 4 5 6 7]


In [66]:
for i in Plist:
    subject = 'P%d'%i
    plotEntropyAnalisis(['C', subject],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),0.4,0.8)
    plotEntropyAnalisis(['C', subject],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),0.4,0.8)
    plotEntropyAnalisisMS(['C', subject],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 0.4, 0.8)

loading the data
loaded the data
loading the data
loaded the data
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
loaded the data
loading the data
loaded the data
loading the data
loaded the data
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
loaded the data
loading the data
loaded the data
loading the data
loaded the data
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
loaded the data
loading the data
loaded the data
loading the data
loaded the data
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
loaded the data
loading the data

## Evolución Control

In [68]:
plotEntropyAnalisis(['C'],[['inicial','final']],'sampen',np.arange(2,20,2),0.4,0.8)
plotEntropyAnalisis(['C'],[['inicial','final']],'permen',np.arange(3,20,2),0.4,0.8)
plotEntropyAnalisisMS(['C'],[['inicial','final']], [2,3], np.arange(2,20,2), 0.4, 0.8)

loading the data
loaded the data
loading the data
loaded the data
2 2
2 4
2 6
2 8
2 10
2 12
2 14
2 16
2 18
3 2
3 4
3 6
3 8
3 10
3 12
3 14
3 16
3 18
loaded the data


### Determinando para qué medidas los controles para cada paciente antes y después son iguales

#### En entropía de la muestra

In [86]:
for metric in np.arange(2,20,2):
    data = pd.read_excel('results/entropy_C_sampen_%d_0.4_0.8_inicialfinal_stats.xlsx'%metric)
    signif = data.groupby(['bands'])['signif'].sum()
    mask = signif < 3
    print(metric,signif[mask])

2 Series([], Name: signif, dtype: int64)
4 Series([], Name: signif, dtype: int64)
6 Series([], Name: signif, dtype: int64)
8 bands
theta    2
Name: signif, dtype: int64
10 bands
gamma    1
Name: signif, dtype: int64
12 bands
beta     2
gamma    0
Name: signif, dtype: int64
14 bands
delta    2
gamma    1
theta    2
Name: signif, dtype: int64
16 bands
beta     2
delta    2
gamma    0
theta    2
Name: signif, dtype: int64
18 bands
delta    1
gamma    0
Name: signif, dtype: int64


### En entropía de permutación

In [87]:
for metric in np.arange(3,20,2):
    data = pd.read_excel('results/entropy_C_permen_%d_0.4_0.8_inicialfinal_stats.xlsx'%metric)
    signif = data.groupby(['bands'])['signif'].sum()
    mask = signif <3
    print(metric,signif[mask])

3 bands
theta    1
Name: signif, dtype: int64
5 bands
theta    1
Name: signif, dtype: int64
7 bands
theta    1
Name: signif, dtype: int64
9 bands
theta    1
Name: signif, dtype: int64
11 bands
theta    1
Name: signif, dtype: int64
13 bands
theta    1
Name: signif, dtype: int64
15 bands
theta    1
Name: signif, dtype: int64
17 bands
theta    2
Name: signif, dtype: int64
19 bands
theta    2
Name: signif, dtype: int64


### Entropía multiescala

In [92]:
for metric in [2,3]:
    for scale in np.arange(2,20,2):
        data = pd.read_excel('results/entropy_C_msen_%d_%d_0.4_0.8_inicialfinal_stats.xlsx'%(metric,scale))
        signif = data.groupby(['bands'])['signif'].sum()
        mask = signif <3
        print(metric,scale,signif[mask])

2 2 Series([], Name: signif, dtype: int64)
2 4 Series([], Name: signif, dtype: int64)
2 6 bands
gamma    2
Name: signif, dtype: int64
2 8 bands
alpha    2
beta     2
delta    2
gamma    2
Name: signif, dtype: int64
2 10 bands
alpha    1
beta     2
delta    2
gamma    2
theta    2
Name: signif, dtype: int64
2 12 bands
all bands    2
alpha        0
beta         2
delta        2
gamma        1
theta        2
Name: signif, dtype: int64
2 14 bands
all bands    1
alpha        1
beta         0
delta        1
gamma        1
theta        1
Name: signif, dtype: int64
2 16 bands
all bands    1
alpha        0
beta         0
delta        1
gamma        1
theta        1
Name: signif, dtype: int64
2 18 bands
all bands    0
alpha        0
beta         0
delta        0
gamma        0
theta        0
Name: signif, dtype: int64
3 2 bands
theta    2
Name: signif, dtype: int64
3 4 Series([], Name: signif, dtype: int64)
3 6 bands
delta    2
gamma    1
Name: signif, dtype: int64
3 8 bands
beta     2
delta    